In [1]:
import numpy as  np
import pandas as pd
import os

In [2]:
# This should point to base dir of regional_forecast git repo
# https://github.com/BayAreaMetro/regional_forecast
REGIONAL_FORECAST_GIT_DIR = r"\\mainmodel\MainModelShare\regional_forecast"

# The location of REMI outputs
REMI_OUTPUT_BOX_DIR       = os.path.join(os.environ["USERPROFILE"],
                            "Box\Modeling and Surveys\Regional Modeling\REMI\REMI files\PBA50 Blueprint\REMI output")

# e.g. Draft Forecast Base Case, Draft Forecast Upper Case, Forecast 03122020
FORECAST_DIR              = "Forecast 005052020"

# e.g. Blueprint Baseline, Blueprint Basic, Blueprint Crossing, Blueprint Plus Fix it First
FORECAST_VARIANT          = "Blueprint Baseline"

# Figure out corresponding UrbanSim Scenario number
# see https://github.com/BayAreaMetro/bayarea_urbansim/blob/zoning_mods_update/configs/scenarios.md
URBANSIM_VARIANT_MAPPING  = {"Blueprint Baseline"         :20,
                             "Blueprint Basic"            :21,
                             "Blueprint Plus Fix it First":22,
                             "Blueprint Crossing"         :23}
URBANSIM_VARIANT          = URBANSIM_VARIANT_MAPPING[FORECAST_VARIANT]

# read employment data
employment_input_file     = os.path.join(REMI_OUTPUT_BOX_DIR, FORECAST_DIR, FORECAST_VARIANT,
                                         "employment_projections_s{}.csv".format(URBANSIM_VARIANT))
employment_output_file    = os.path.join(REMI_OUTPUT_BOX_DIR, FORECAST_DIR, FORECAST_VARIANT, 
                                         "emp by ind11_s{}.csv".format(URBANSIM_VARIANT))

In [4]:
#Map the relationship between ACS sectors and REMI sectors.
#We use ACS sector aggregration because self employment data comes from ACS Table C24070, which uses this aggregration
sectormap = pd.read_csv(os.path.join(REGIONAL_FORECAST_GIT_DIR, "regionals post process", "sectormap.csv"))

#remi BEA concept employment translates to EDD+ACS concept with a adjustment factor calculated using 2015 data
emp_translate = pd.read_csv(os.path.join(REGIONAL_FORECAST_GIT_DIR, "regionals post process", "emp_translate.csv"))

IOError: File \\mainmodel\MainModelShare\regional_forecast\regionals post process\sectormap.csv does not exist

In [ ]:
sectormap.head()

In [ ]:
emp_translate.head()

In [ ]:
emp = pd.read_csv(employment_input_file)

# merge with sectormap
emp_merge = pd.merge(sectormap,emp, how='inner', on=['Industries'])

In [ ]:
emp_merge

In [ ]:
emp_sum = emp_merge.groupby(['ind']).sum().reset_index()

In [ ]:
emp_sum

In [ ]:
emp_merge2 = pd.merge(emp_sum,emp_translate, how='inner', on=['ind']).set_index('ind')

In [ ]:
emp_merge2

In [ ]:
#choose only column needed and multiply by adjustment
emp_merge2 = emp_merge2[['2015','2020','2025','2030','2035','2040','2045','2050']].mul(emp_merge2.adjustment, axis = 0)
#round to int and bring industry back to it's own column
emp_merge3 = emp_merge2.astype(int).reset_index()

In [ ]:
emp_merge3

In [ ]:
emp_merge3.to_csv(employment_output_file, header=True, index=False)